In [ ]:
import torch
import torchvision
from torchvision import transforms as T
import torchvision.datasets as dset
import torch.nn.functional as F
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import torchvision.utils as vutils
import os
import random
from PIL import Image
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/Colab Notebooks/dataset.zip'

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
  inflating: train/0350_c1_083624030.jpg  
  inflating: train/1411_c1_004276123.jpg  
  inflating: train/1411_c1_063612070.jpg  
  inflating: train/1411_c3_073654330.jpg  
  inflating: train/1411_c5_065380551.jpg  
  inflating: train/0959_c1_091954569.jpg  
  inflating: train/0959_c3_014933988.jpg  
  inflating: train/0959_c5_006154919.jpg  
  inflating: train/0959_c6_045481865.jpg  
  inflating: train/0852_c6_004791953.jpg  
  inflating: train/1177_c3_011636446.jpg  
  inflating: train/0474_c1_043109826.jpg  
  inflating: train/0474_c5_069409220.jpg  
  inflating: train/0474_c5_013563167.jpg  
  inflating: train/0474_c5_029799244.jpg  
  inflating: train/1487_c1_003967228.jpg  
  inflating: train/1487_c5_046010440.jpg  
  inflating: train/1487_c5_044022610.jpg  
  inflating: train/1487_c6_050046539.jpg  
  inflating: train/1422_c3_032260579.jpg  
  inflating: train/1193_c1_030694336.jpg  
  inflating: train/1225_c6_035586085.jpg

In [ ]:

train_folder = "/content/train/"
train_group_folder = "/content/train_group/"
check_point_path="/content/drive/MyDrive/Colab Notebooks/check_point/" 

num_epochs = 100 


In [ ]:
def initialize_alexnet(num_classes):
  # load the pre-trained Alexnet
  alexnet = torchvision.models.alexnet(pretrained=True)
  
  # get the number of neurons in the penultimate layer
  in_features = alexnet.classifier[6].in_features
  
  # re-initalize the output layer
  alexnet.classifier[6] = torch.nn.Linear(in_features=in_features, 
                                          out_features=num_classes)
  
  return alexnet

In [ ]:
def initialize_resnet(num_classes):
  # load the pre-trained Alexnet
  resnet =  torchvision.models.resnet50(pretrained=True)
  # get the number of neurons in the penultimate layer
  in_features = resnet.fc.in_features
  
  # re-initalize the output layer
  resnet.fc = torch.nn.Linear(in_features=in_features, 
                                          out_features=num_classes)
  
  return resnet

In [ ]:
def get_cost_function():
  cost_function = torch.nn.TripletMarginLoss(margin=1.0, p=2)
  return cost_function

In [ ]:
def get_optimizer(net, lr, wd, momentum):
  optimizer = torch.optim.SGD(net.parameters(), lr=lr, weight_decay=wd, momentum=momentum)
  return optimizer

In [ ]:
test_folder = "/content/test/"
query_folder = "/content/queries/"
net_type="resnet50"
device='cuda:0'
learning_rate=0.01
weight_decay=0.000001
momentum=0.9

In [ ]:
# Prepare data transformations and then combine them sequentially
transform = list()
#transform.append(T.ToTensor())                            # converts Numpy to Pytorch Tensor
#transform.append(T.Normalize(mean=[0.5], std=[0.5]))      # Normalizes the Tensors between [-1, 1]
#transform = T.Compose(transform)                          # Composes the above transformations into one.



transform.append(T.Resize((256, 256)))                      # Resize each PIL image to 256 x 256
transform.append(T.RandomCrop((224, 224)))                  # Randonly crop a 224 x 224 patch
transform.append(T.ToTensor())                              # converts Numpy to Pytorch Tensor
transform.append(T.Normalize(mean=[0.485, 0.456, 0.406], 
                              std=[0.229, 0.224, 0.225]))    # Normalize with ImageNet mean
transform = T.Compose(transform)
checkpoint = torch.load(check_point_path+net_type+"_max"+".pt")
net =initialize_resnet(512).to(device)
optimizer = get_optimizer(net, learning_rate, weight_decay, momentum)
net.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['val_loss']
print(epoch)
print(loss)
test_emb={}



with torch.no_grad():
  for filename in os.listdir(test_folder):
    img=Image.open(os.path.join(test_folder,filename))
    tensor_img=transform(img).to(device)
    tensor_img.unsqueeze_(0)
    output=net(tensor_img).cpu().numpy()
    test_emb[filename]=output




58
0.0008808102856079738


In [ ]:
def get_embedding(net,transform,path):
  img=Image.open(path)
  #img=Image.open(filename)
  tensor_img=transform(img).to(device)
  tensor_img.unsqueeze_(0)
  output=net(tensor_img).cpu().numpy()
  return output

def find_k_neighbor(example,dict_emb, k):
  list_value=[]
  list_filename=[]
  for i in range(k):
    list_value.append(100000)
    list_filename.append("x")
  min=1000
  for x in dict_emb.keys():
    #value=torch.cdist(dict_emb[x],example)
    value=np.linalg.norm(dict_emb[x]-example)
    if value<min:
      min=value
    for idx,v in enumerate(list_value):
      if value<v: 
        list_value.insert(idx, value)
        list_filename.insert(idx,x)
        del list_value[-1]
        del list_filename[-1]
        break
  return list_filename,list_value
def  k_reciprocal_reranking(net, transform, example,dict_emb,k,list_knn, list_knn_value):
  final_list=[]
  for idx,candidate in enumerate(list_knn):
    path = os.path.join(test_folder,filename)
    candidate_emb=get_embedding(net, transform,path)
    (list_neibor,list_neibor_value) = find_k_neighbor(candidate_emb,dict_emb,k+1)
    for nei in list_neibor_value:
      if list_knn_value[idx] < nei:
        final_list.append(candidate)
        break
  return final_list


i=0
with open('result.txt', 'w') as f:
    
  with torch.no_grad():
    for filename in os.listdir(query_folder):
      f.write(filename+": ")
      #filename="/content/test/000000.jpg"
      path = os.path.join(query_folder,filename)
      output=get_embedding(net, transform, path)
      list_knn, list_knn_value= find_k_neighbor(output,test_emb,10)
      final_list= k_reciprocal_reranking(net, transform, output,test_emb,10,list_knn, list_knn_value)
      print(final_list)
      for idx,re in  enumerate(final_list):
        if idx==len(final_list)-1:
          f.write(re)
          f.write("\n")  
        else: 
          f.write(re+", ")
      i=i+1
      print(i)
f.close()

['006626.jpg', '012796.jpg', '009563.jpg', '012379.jpg', '013667.jpg', '014301.jpg', '016582.jpg', '003963.jpg', '015141.jpg']
1
['006239.jpg', '008756.jpg', '017693.jpg', '008783.jpg', '015046.jpg', '003702.jpg', '005867.jpg', '013544.jpg', '018424.jpg', '008694.jpg']
2
['005305.jpg', '006826.jpg']
3
['003056.jpg', '007293.jpg', '014329.jpg', '014040.jpg', '009068.jpg', '009352.jpg', '000698.jpg', '017150.jpg', '012124.jpg', '007626.jpg']
4
['018335.jpg', '015051.jpg', '016758.jpg', '012609.jpg', '013489.jpg', '003565.jpg']
5
['003682.jpg', '011989.jpg', '001251.jpg', '006139.jpg', '000853.jpg', '004707.jpg']
6
['009041.jpg', '001957.jpg']
7
['002569.jpg', '006416.jpg', '015436.jpg', '017228.jpg', '009141.jpg']
8
['015791.jpg']
9
['008558.jpg', '004211.jpg']
10
['006701.jpg', '018938.jpg', '009632.jpg']
11
['018674.jpg', '013849.jpg', '017643.jpg', '012124.jpg', '012673.jpg', '002766.jpg', '000405.jpg', '000503.jpg', '012356.jpg', '002328.jpg']
12
['006632.jpg', '004410.jpg', '018838.